In [4]:
import os
import io
import requests
from PIL import Image
import PyPDF2
from pdf2image import convert_from_path
from openai import OpenAI
from pathlib import Path
import dotenv
import base64

dotenv.load_dotenv("./streamlit_local/.env")
# Get the directory of the current script (e.g., app.py)
# SCRIPT_DIR = Path(__file__).parent.resolve()


True

In [ ]:

def pdf_to_image(pdf_path, page_number, output_folder="temp"):
    """
    Convertit une page spécifique d'un PDF en image
    """
    # Créer le dossier temporaire s'il n'existe pas
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Convertir la page spécifique en image
    images = convert_from_path(pdf_path, first_page=page_number, last_page=page_number)
    
    if not images:
        raise ValueError(f"Aucune image trouvée pour la page {page_number}")
    
    # Sauvegarder l'image temporaire
    image_path = os.path.join(output_folder, f"page_{page_number}.jpg")
    images[0].save(image_path, "JPEG")
    
    return image_path

def describe_image_with_openai(image_path):
    """
    Envoie une image à l'API OpenAI pour en obtenir une description
    """
    client = OpenAI()
    
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Describe this image"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_file.read().encode('base64')}"
                            },
                        },
                    ],
                }
            ],
            temperature=0,
        )
    
    return response.choices[0].message.content


In [3]:

def main():
    # Configuration
    pdf_path = "./data/PU_P01_AAP07.pdf"
    page_number = 3
    # api_key = input("Entrez votre clé API OpenAI: ")
    
    try:
        # Étape 1: Convertir la page PDF en image
        print("Conversion de la page PDF en image...")
        image_path = pdf_to_image(pdf_path, page_number)
        
        # Étape 2: Envoyer l'image à OpenAI pour description
        print("Envoi de l'image à OpenAI pour description...")
        description = describe_image_with_openai(image_path)
        
        # Afficher le résultat
        print("\nDescription de la page:")
        print(description)
        
    except Exception as e:
        print(f"Une erreur s'est produite: {str(e)}")
    # finally:
        # Nettoyage: supprimer l'image temporaire
        # if os.path.exists(image_path):
        #     os.remove(image_path)

if __name__ == "__main__":
    main()

Conversion de la page PDF en image...
Une erreur s'est produite: name 'pdf_to_image' is not defined


In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

image_path="./temp/image_form.png"

with open(image_path, "rb") as image_file:
    # Encoder l'image en base64
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

system_prompt="""
            I have a collection of form images that include questions in different configurations. The forms may include:

            Straight and direct interrogative questions:
            Examples:

            "Where is the project located?"
            "Who are the target beneficiaries?"
            Informational expectations presented in a descriptive (non-interrogative) style:
            Examples:

            "Project description"
            "Expected outcomes"
            Questions that include an explanation, sub-questions, or multiple choice items as guidance:
            Examples:

            Example 1: Project Justification and Beneficiary
            "Problem Analysis (max. 600 words)
            Please provide:
            a description of the current situation or issue related to the project (background, geographic region, and beneficiaries, etc).
            an analysis of the problem the project is trying to address. Develop a Problem Tree by defining the core problem and identifying its causes and effects. The description must be clearly linked to the project’s objectives and how the project will address the problem."
            Example 2: Date of Submission
            "When will the project be submitted to the donor?
            Is the submission date aligned with the donor’s deadlines?
            Are there any internal deadlines for reviews before submission?
            Would a timeline graphic of submission milestones be helpful?"
            Example 3: Blue Planet Fund outcomes
            "Which Blue Planet Fund outcome(s) does your project address? Select all that apply.
            □ Marine Protected Areas (MPAs) and Other Effective Conservation Measures (OECMs)
            □ Illegal, Unreported, and Unregulated Fishing (IUU)
            □ International and large-scale fisheries
            □ Solid waste and other forms of marine pollution
            □ Critical marine habitats for coastal resilience
            □ Small-scale fisheries management
            □ Aquaculture
            □ None of the above. If so, please tell us how you address protecting the marine environment."
            Your task is to process the provided images of these forms, extract all the questions (regardless of their configuration), and output a JSON-like list of strings. Each string in the list should represent one question or one key informational prompt extracted from the image.

            The expected output format should be like this:
            [
                "Question 1 extracted from the image + multiple choice options +  relevant information notes",
                "Question 2 extracted from the image + multiple choice options +  relevant information notes",
                "Question 3 extracted from the image + multiple choice options +  relevant information notes"
                ...
            ]

            Make sure to:
            Accurately extract and capture the essence of the question or informational prompt from every form, even if the wording is not in a strictly interrogative style.
            Include  multiple choice items, sub-questions or any guidance details if they are integral to the overall question prompt.
            Preserve the order of appearance as found in the images (if applicable) or indicate if the order is arbitrary.

        """


model="meta-llama/llama-4-maverick:free"
model="mistralai/mistral-small-3.1-24b-instruct:free"
completion = client.chat.completions.create(  
  model=model,
  messages=[
      {"role": "system", "content": system_prompt},
      {
        "role": "user",
        "content": [
            {"type": "text", "text": "Extract the questions from this form"},
            {
                "type": "image_url",
                "image_url": {                                
                    "url": f"data:image/jpeg;base64,{base64_image}"
                },
            },
        ],
      }
  ],
  temperature=0,
)
print(completion.choices[0].message.content)

Here is the extracted list of questions and informational prompts from the provided form image:

```json
[
    "Q2.1 Project title (Max 10 words)",
    "Q2.2 Project summary NOTE: this summary may be published on the OCEAN website and used to promote your project if successful. (Max 150 words)",
    "Q2.3 Blue Planet Fund outcomes Which Blue Planet Fund outcome(s) does your project address? Select all that apply. 
        □ Marine Protected Areas (MPAs) and Other Effective Conservation Measures (OECMs)
        □ Illegal, Unreported, and Unregulated Fishing (IUU)
        □ International and large-scale fisheries
        □ Solid waste and other forms of marine pollution
        □ Critical marine habitats for coastal resilience
        □ Small-scale fisheries management
        □ Aquaculture
        □ None of the above. If so, please tell us how you address protecting the marine environment.",
    "Q2.4 Marine ecosystems Does your project have a direct focus on any of the following marine

In [19]:
resp=completion.choices[0].message.content
start=resp.find('```json')+7
end=resp.find("""]
```""")+1
resp=resp[start: end]
print(resp)

import json

#json.loads(resp)


[
    "Q2.1 Project title (Max 10 words)",
    "Q2.2 Project summary NOTE: this summary may be published on the OCEAN website and used to promote your project if successful. (Max 150 words)",
    "Q2.3 Blue Planet Fund outcomes Which Blue Planet Fund outcome(s) does your project address? Select all that apply. 
        □ Marine Protected Areas (MPAs) and Other Effective Conservation Measures (OECMs)
        □ Illegal, Unreported, and Unregulated Fishing (IUU)
        □ International and large-scale fisheries
        □ Solid waste and other forms of marine pollution
        □ Critical marine habitats for coastal resilience
        □ Small-scale fisheries management
        □ Aquaculture
        □ None of the above. If so, please tell us how you address protecting the marine environment.",
    "Q2.4 Marine ecosystems Does your project have a direct focus on any of the following marine ecosystem(s)? Select all that apply.
        □ Coral reefs
        □ Intertidal forests and shrublands 

In [20]:
questions=[
    "Q2.1 Project title (Max 10 words)",
    "Q2.2 Project summary NOTE: this summary may be published on the OCEAN website and used to promote your project if successful. (Max 150 words)",
    """Q2.3 Blue Planet Fund outcomes Which Blue Planet Fund outcome(s) does your project address? Select all that apply. 
        □ Marine Protected Areas (MPAs) and Other Effective Conservation Measures (OECMs)
        □ Illegal, Unreported, and Unregulated Fishing (IUU)
        □ International and large-scale fisheries
        □ Solid waste and other forms of marine pollution
        □ Critical marine habitats for coastal resilience
        □ Small-scale fisheries management
        □ Aquaculture
        □ None of the above. If so, please tell us how you address protecting the marine environment.""",
    """Q2.4 Marine ecosystems Does your project have a direct focus on any of the following marine ecosystem(s)? Select all that apply.
        □ Coral reefs
        □ Intertidal forests and shrublands (e.g., mangroves)
        □ Seagrass meadows
        □ Kelp forests
        □ Shellfish beds & reefs
        □ Coastal inlets, riverine estuaries and bays, coastal lakes and lagoons
        □ Coastal salt marsh or reedbed
        □ Ocean waters
        □ Deep sea floors
        □ Anthropogenic marine biome (artificial structures, marine aquafarms)
        □ Shorelines (rocky, muddy, sandy, boulder & cobble)
        □ Coastal Shrublands and Grasslands
        □ No direct focus on specific ecosystem(s)""",
    "Q2.5 Project region What region will your project work in? Select all that apply."
]

In [9]:
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
from openai import OpenAI
import dotenv

dotenv.load_dotenv("./streamlit_local/.env")

def compare_with_cosine_similarity(model, text1, text2):    

    # encode queries and passages
    text1_embeddings = model.encode([text1], normalize_embeddings=True)
    text2_embeddings = model.encode([text2], normalize_embeddings=True)

    # calculate cosine similarity
    return util.cos_sim(text1_embeddings, text2_embeddings)


def compare_with_llm(text1, text2):
    client = OpenAI()
        
    system_prompt="""
        You are a semantic duplicate detector. Given two input texts A and B, determine whether they express the same meaning and therefore should be considered duplicates.  

        • Your answer must include:
        1. a label: SAME” or “NOT_SAME
        2. a confidence score from 0.0 (no confidence) to 1.0 (maximum confidence)        

        • Consider variations such as:
        – truncation (one text is an excerpt of the other)
        – paraphrase (different wording but same intent)
        – re‑ordering or added optional detail
        – synonyms, but pay attention to changes in scope or emphasis
        – differences in question verbs or focus words that alter meaning

        • Strictly label as SAME only if their core meaning and intent coincide—even if one is shorter, truncated, or worded differently.

        Output a json schema

    """
    user_prompt=f"""
        Text 1: {text1}
        Text 2: {text2}
    """

    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {
                    "role": "system",
                    "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],        
    )

    return response.choices[0].message.content

def compare_with_fuzzy(text1, text2):
    return fuzz.token_set_ratio(text1[:200], text2[:200])            


minilm= SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", trust_remote_code=True, device="cpu"),




In [10]:
src_text="""
    I have read the Guidance, including the “Complete Guidance for 
    Applicants”, “Gender Equality, Disability and Social Inclusion 
    Guidance”, “Finance Guidance” and “Monitoring, Evaluation and 
    Learning Guidance”
"""

extracted_text="""
    I have read the Guidance, including the “Complete Guidance for Applicants”, 
    “Gender Equality, Disability and Social Inclusion Guidance”, “Finance 
    Guidance” and “Monitoring, Evaluation and Learning Guidance”
"""

print(compare_with_cosine_similarity(model=minilm[0], text1=src_text, text2=extracted_text))
print(compare_with_fuzzy(src_text, extracted_text))
print(compare_with_llm(src_text, extracted_text))

tensor([[1.0000]])
100
{"label":"SAME","confidence":1.0}
